In [30]:
"""
 author:jjk
 datetime:2019/11/26
 coding:utf-8
 project name:Pycharm_workstation
 Program function: jieba分词操作详解
 
"""
"""

支持三种分词模式与特点：
    精确模式：试图将句子最精确地切开，适合文本分析；
    全模式：把句子中所有的可以成词的词语都扫描出来，速度非常快，但是不能解决歧义；
    搜索引擎模式：在精确模式的基础上对长词再次切分，提高召回率，适用于搜索引擎。
    支持繁体分词
    支持自定义词典
安装方法：
    自动安装：pip install jieba
    半自动安装：下载http://pypi.python.org/pypi/jieba/解压后运行python setup.py install
    手动安装：将jieba 目录放置于当前目录或者site-packages目录，通过 import jieba 来引用
核心算法：
    基于前缀词典实现高效的词图扫描，生成句子中汉字所有可能成词情况所构成的有向无环图（DAG）
    采用了动态规划查找最大概率路径，找出基于词频的最大切分组合，对于未登录词，采用了基于汉字成词能力的HMM模型，使用了Viterbi算法

主要功能：
    jieba.cut三个输入参数：待分词的字符串；cut_a11参数是否全模式；HMM参数是否是HMM模型
    jieba.cut_for_search两个参数：待分词的字符串；是否HMM模型。该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细
    jieba.lcut以及 jieba.lcut_for_search直接返回list
    jieba.Tokenizer（dictionary=DEFAULT_DICT）新建自定义分词器，可用于同时使用不同词典。 
    jieba.dt为默认分词器
    
"""
import jieba,os,sys

In [31]:
#********************1 结巴中文分词基本操作***********************************
# 1、全模式、扫描所有可能成词的词语，速度非常快，不能解决歧义问题
print('='*40)
print('1.分词')
print('-'*40)
seg_list = jieba.cut('贾继康来自昆明理工大学',cut_all=True) # cut_all=True采用全模式
print('Full Mode:' + ','.join(seg_list)) # 全模式就是将所有可能构成词的分割开

1.分词
----------------------------------------
Full Mode:贾,继,康,来自,昆明,明理,理工,理工大,理工大学,工大,大学


In [32]:
# 2、默认是精确模式，适合文本分析
print('='*40)
print('1.分词')
print('-'*40)
seg_list = jieba.cut('贾继康来自昆明理工大学',cut_all=False) # 默认的分词方式
print('Default Mode:' + ','.join(seg_list))

1.分词
----------------------------------------
Default Mode:贾继康,来自,昆明,理工大学


In [33]:
# 3、搜索引擎模式，对长词再次切分，提高召回率，适合用于搜索引擎分词
# 
print('='*40)
print('1.分词')
print('-'*40)
seg_list = jieba.cut_for_search('贾继康来自昆明理工大学',HMM=False)
print('搜索引擎方式：' + ','.join(seg_list))

1.分词
----------------------------------------
搜索引擎方式：贾,继,康,来自,昆明,理工,工大,大学,理工大,理工大学


In [39]:
print('='*40)
print('2.添加自定义词典/调整词典')
print('-'*40)

seg_list = jieba.cut('如果放到数据库中将出错',HMM=False)
# 如果/放到/数据库/中/将/出错
print('原文档:' + '/'.join(seg_list))

seg_list = jieba.cut('如果放到数据库中将出错',HMM=False)
print(jieba.suggest_freq(('中','将'),True)) # 调整词典-实现‘中将拆分’
print('改进文档:' + '/'.join(seg_list))

# 合成
print('\n原文档：\t'+'/'.join(jieba.cut('「台中」正确应该不会被切开', HMM=False))) #「/台/中/」/正确/应该/不会/被/切开
print(jieba.suggest_freq('台中', True))
print('改进文档：\t'+'/'.join(jieba.cut('「台中」正确应该不会被切开', HMM=False))) #「/台中/」/正确/应该/不会/被/切开


2.添加自定义词典/调整词典
----------------------------------------
原文档:如果/放到/数据库/中/将/出错
488
改进文档:如果/放到/数据库/中/将/出错

原文档：	「/台中/」/正确/应该/不会/被/切开
70
改进文档：	「/台中/」/正确/应该/不会/被/切开


In [42]:
#********************2 加载自定义分词词典***********************************
#sys.path.append(../)
jieba.load_userdict("../dataSet/StopWord/user_dict.txt") # 加载词典

seg_list1 = jieba.cut("今天很高兴在慕课网和大家交流学习")
print('\n\n加载自定义分词词典：\n' + "/ ".join(seg_list1))




加载自定义分词词典：
今天很高兴/ 在/ 慕课网/ 和/ 大家/ 交流学习


In [49]:
print('='*40)
print('3. 关键词提取')
print('-'*40)
print(' TF-IDF')
print('-'*40)

'''
extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
sentence 为待提取的文本
topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
withWeight 为是否一并返回关键词权重值，默认值为 False
allowPOS 仅包括指定词性的词，默认值为空，即不筛选
jieba.analyse.TFIDF(idf_path=None) 新建 TFIDF 实例，idf_path 为 IDF 频率文件
'''
import jieba.analyse
s = "此外，公司拟对全资子公司吉林欧亚置业有限公司增资4.3亿元，增资后，吉林欧亚置业注册资本由7000万元增加到5亿元。吉林欧亚置业主要经营范围为房地产开发及百货零售等业务。目前在建吉林欧亚城市商业综合体项目。2013年，实现营业收入0万元，实现净利润-139.13万元。"
for x, w in jieba.analyse.extract_tags(s,10, withWeight=True):
    print('%s %s' % (x, w)) 

print('-'*40)
print(' TextRank')
print('-'*40)
# 文本排名
for x, w in jieba.analyse.textrank(s, 10,withWeight=True):
    print('%s %s' % (x, w))


3. 关键词提取
----------------------------------------
 TF-IDF
----------------------------------------
欧亚 0.7300142700289363
吉林 0.659038184373617
置业 0.4887134522112766
万元 0.3392722481859574
增资 0.33582401985234045
4.3 0.25435675538085106
7000 0.25435675538085106
2013 0.25435675538085106
139.13 0.25435675538085106
实现 0.19900979900382978
----------------------------------------
 TextRank
----------------------------------------
吉林 1.0
欧亚 0.9966893354178172
置业 0.6434360313092776
实现 0.5898606692859626
收入 0.43677859947991454
增资 0.4099900531283276
子公司 0.35678295947672795
城市 0.34971383667403655
商业 0.34817220716026936
业务 0.3092230992619838


In [58]:
import jieba.posseg
print('='*40)
print('4. 词性标注')
print('-'*40)

words = jieba.posseg.cut("我爱北京天安门")
for word, flag in words:
    print('%s %s' % (word, flag))

print('='*40)
print('6. Tokenize: 返回词语在原文的起止位置')
print('-'*40)
print(' 默认模式')
print('-'*40)

result = jieba.tokenize('永和服装饰品有限公司')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

print('-'*40)
print(' 搜索模式')
print('-'*40)

result = jieba.tokenize('永和服装饰品有限公司', mode='search')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

4. 词性标注
----------------------------------------
我 r
爱 v
北京 ns
天安门 ns
6. Tokenize: 返回词语在原文的起止位置
----------------------------------------
 默认模式
----------------------------------------
word 永和		 start: 0 		 end:2
word 服装		 start: 2 		 end:4
word 饰品		 start: 4 		 end:6
word 有限公司		 start: 6 		 end:10
----------------------------------------
 搜索模式
----------------------------------------
word 永和		 start: 0 		 end:2
word 服装		 start: 2 		 end:4
word 饰品		 start: 4 		 end:6
word 有限		 start: 6 		 end:8
word 公司		 start: 8 		 end:10
word 有限公司		 start: 6 		 end:10
